In [ ]:
# Pygame installation
!pip install pygame

In [1]:
import pygame as pg
from pygame import image as img

pygame 2.1.0 (SDL 2.0.16, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Function definitions for use
def load_image(file):
    """loads an image, prepares it for play"""
    try:
        surface = pg.image.load(file)
    except pg.error:
        raise SystemExit('Could not load image "%s" %s' % (file, pg.get_error()))
    return surface.convert_alpha() # convert_alpha allows for transparency from .pngs

In [3]:
pg.init()
win = pg.display.set_mode((0,0))
pg.display.set_caption("Custom Environment - Guess Path")

class Robot(pg.sprite.Sprite):
    """Visual for AI space and movement direction"""
    images = [load_image("assets/sprites/robo_R.png"), 
              load_image("assets/sprites/robo_L.png"), 
              load_image("assets/sprites/robo_U.png"),
              load_image("assets/sprites/robo_D.png")]

    def __init__(self, pos):
        pg.sprite.Sprite.__init__(self, self.containers)
        self.image = self.images[0]
        self.rect = self.image.get_rect()
        self.rect.x = pos[0] - (self.rect.width / 2.0)
        self.rect.y = pos[1] - (self.rect.height / 2.0)

    def update(self, action: int):
        if action >= 0:
            self.image = self.images[action]
    
    def setPosition(self, newPos):
        self.rect = self.image.get_rect()
        self.rect.x = newPos[0] - (self.rect.width / 2.0)
        self.rect.y = newPos[1] - (self.rect.height / 2.0)

class Space(pg.sprite.Sprite):
    """Visual for space type. Normal, positive or negative"""
    images = [load_image("assets/sprites/normal.png"), 
              load_image("assets/sprites/plus.png"), 
              load_image("assets/sprites/minus.png"),
              load_image("assets/sprites/start.png"), 
              load_image("assets/sprites/goal.png")]

    def __init__(self, pos, jumpDistance):
        pg.sprite.Sprite.__init__(self, self.containers)
        
        self.jumpDistance = jumpDistance
        
        if(jumpDistance == 0):  # neutral movement - does not push Robot
            self.image = self.images[0]
        elif(jumpDistance > 0): # positive movement - pushes robot forward
            self.image = self.images[1]
        else:                    # negative movement - pushes robot backward
            self.image = self.images[2]

        self.rect = self.image.get_rect()
        self.rect.x = pos[0] - (self.rect.width / 2.0)
        self.rect.y = pos[1] - (self.rect.height / 2.0)
        
    def setType(self, spaceType):
            self.image = self.images[spaceType]
            
            if spaceType == 1:
                self.jumpDistance = 2
            elif spaceType == 2:
                self.jumpDistance = -2

# Initialize Game Groups
all = pg.sprite.RenderUpdates()
    
Space.containers = all
Robot.containers = all

# Board does not require rendering
        
class Board():
    """Board that sets up and displays all spaces"""
    def __init__(self, maxCols, pos):
        self.spaces = [[], [], [], [], []]
        
        self.maxCols = maxCols
        
        if self.maxCols < 1:
            self.maxCols = 1 # cap minimum value to max rows in case of emergency
        
        self.middle = (len(self.spaces) // 2)
        self.playerPos = [0,self.middle]
        self.goal = [self.middle, self.maxCols - 1]
        
        # Thanks to Jack Malone for help with this loop
        for index, value in enumerate(self.spaces):
                    for y in range(self.maxCols):
                            value.append(Space(((64 * y) + pos[0], (64 * index) + pos[1]), 0))
                        
    def setStartEnd(self):
        self.spaces[self.playerPos[1]][self.playerPos[0]].setType(3)
        self.spaces[self.goal[0]][self.goal[1]].setType(4)
                        
    def update(self, robot: Robot, action: int):
        if action >= 0:
            if action == 0: # right
                  if self.playerPos[0] < self.maxCols - 1:
                        self.playerPos[0] += 1
            elif action == 1: # left
                 if self.playerPos[0] > 0:
                        self.playerPos[0] -= 1
            elif action == 2: # up
                if self.playerPos[1] > 0:
                        self.playerPos[1] -= 1
            elif action == 3: # down
                if self.playerPos[1] < len(self.spaces) - 1:
                        self.playerPos[1] += 1
            
            # first check to see if the position has to be updated
            # as the robot may move onto a push space
            
            pushValue = self.spaces[self.playerPos[1]][self.playerPos[0]].jumpDistance
            
            if pushValue != 0: # only do push calculations if the robot has to be pushed
                self.playerPos[0] += pushValue

                # now check to see if the player has jumped outside the bounds of the board
                if self.playerPos[0] > self.maxCols - 1:
                    self.playerPos[0] = self.maxCols - 1
                elif self.playerPos[0] < 0:
                    self.playerPos[0] = 0
                
            # now that the robot has moved, update it's position
            robot.setPosition(((self.playerPos[0] * 64) + 96, (self.playerPos[1] * 64) + 96))
            
    def initializeSpaces(self):
        self.spaces[self.middle][3].setType(1)
        self.spaces[0][4].setType(2)
        self.spaces[1][4].setType(2)
        self.spaces[2][4].setType(2)
        self.spaces[3][4].setType(2)
        self.spaces[4][4].setType(2)
        
        self.spaces[0][7].setType(1)
        self.spaces[0][8].setType(2)
        self.spaces[1][8].setType(2)
        self.spaces[2][8].setType(2)
        self.spaces[3][8].setType(2)
        self.spaces[4][8].setType(2)
        
        self.spaces[3][11].setType(1)
        self.spaces[0][12].setType(2)
        self.spaces[1][12].setType(2)
        self.spaces[2][12].setType(2)
        self.spaces[3][12].setType(2)
        self.spaces[4][12].setType(2)
        
    def winCheck(self):
        if(self.playerPos[1] == self.goal[0] 
           and self.playerPos[0] == self.goal[1]):
            return True
        return False

pg.quit()



In [4]:
# Game Setup
pg.init()
win = pg.display.set_mode((1150,500))
pg.display.set_caption("Custom Env")
bg = load_image('assets/sprites/background.png')
board = Board(16, (100,100))
board.setStartEnd()
robo = Robot(((board.playerPos[0] * 64) + 96, 
              (board.playerPos[1] * 64) + 96))
movesMade = 0

GAME_FONT = pg.freetype.Font("assets/fonts/arial.ttf", 24)

board.initializeSpaces()

action = -1
clock = pg.time.Clock()

run = True

def draw():
    win.blit(bg, (0,0))
    
    for r in board.spaces:
            for c in r: # position all spaces to the correct top left position
                win.blit(c.image, (c.rect.x, c.rect.y))
                
    win.blit(robo.image, (robo.rect.x, robo.rect.y))
    
    GAME_FONT.render_to(win, (50,50), "Steps taken: " + str(movesMade), (250, 250, 250))

    pg.display.update()

while run:
    clock.tick(30)
    
    for event in pg.event.get():
        if event.type == pg.QUIT:
            run = False
        elif event.type == pg.KEYUP :
            if keys[pg.K_RIGHT]:
                action = 0
                movesMade += 1
            elif keys[pg.K_LEFT]:
                action = 1
                movesMade += 1
            elif keys[pg.K_UP]:
                action = 2
                movesMade += 1
            elif keys[pg.K_DOWN]:
                action = 3
                movesMade += 1
            
            
    keys = pg.key.get_pressed()

    
    robo.update(action)
    board.update(robo, action)
    
    action = -1;
    
    draw()
    
    if(board.winCheck()):
        run = False
        
    
pg.quit()

In [27]:
pg.quit()